# Part I

In [19]:
import pyaudio
import numpy as np
CHUNK = 1024
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
record_time=5
Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
frames=[]
for i in range(0,Expected_num_of_chunks): 
    data = stream.read(CHUNK)
    frames.append(data)      
array_frames=np.array(frames)
intframes=np.frombuffer(array_frames, np.int16)
stream.stop_stream()
stream.close()
p.terminate()
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)

# Part II

In [31]:
import keyboard
import time
import wave
import pyaudio
import numpy as np
CHUNK = 1600
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
def hit_to_talk():
    keyboard.wait('s')
    time.sleep(1)
    print("Please talk")
def EnergyPerSampleInDecibel(audioframe):
    audioframe=np.frombuffer(np.array(audioframe),np.int16)
    np_audioframe=np.array(audioframe,dtype='int64')
    energy= 10*np.log10(np.sum(np.square(np_audioframe)))
    return energy
def find_ini_background(audioframe):
    background_energy=0
    for i in range(100):
        background_energy+=EnergyPerSampleInDecibel(audioframe)
    return background_energy/100
def classifyFrame(audioframe,background):
    forgetfactor=1
    isSpeech=False
    current = EnergyPerSampleInDecibel(audioframe)
    level=EnergyPerSampleInDecibel(audioframe)
    threshold=10
    adjustment=0.05
    print('Background',background,'current',current)
    if current < background:
        background = current
    else:
        background+=(current - background)*adjustment
    if (level < background): level = background
    if (level - background > threshold): isSpeech = True
    return isSpeech
def record():
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    stop_record_time=0.7
    stop_speak_time=0
    stop_speak_per_loop=0.1
    isSpeech=False
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    frames=[]
    intframes=[]
    First_run=True
    while True:
        audioframe=stream.read(CHUNK)
        if First_run:
            background=find_ini_background(audioframe)
            First_run=False
        frames.append(audioframe)
        np_array=np.array(audioframe)
        int_np_array=np.frombuffer(np_array, np.int16)
        intframes+=int_np_array.tolist()
        isSpeech=classifyFrame(audioframe,background)
        if isSpeech==True:
            stop_speak_time=0
        else:
            stop_speak_time+=0.1
        if stop_speak_time>=stop_record_time:
            wf = wave.open('1.wav', 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
            wf.setframerate(sampling_rate)
            wf.writeframes(b''.join(frames))
            wf.close()
            print("Audio data saved as 1.wav")
            stream.stop_stream()
            stream.close()
            p.terminate()
            break
def start():
    hit_to_talk()
    record()

In [33]:
a=start()

Please talk
Background 66.59721056327747 current 66.5972105632775
Background 66.59721056327747 current 83.95299661727174
Background 66.59721056327747 current 74.91935396004712
Background 66.59721056327747 current 77.98559192317411
Background 66.59721056327747 current 106.44485411442297
Background 66.59721056327747 current 117.11748739982171
Background 66.59721056327747 current 111.95924399922617
Background 66.59721056327747 current 105.5238053969836
Background 66.59721056327747 current 114.01905556024994
Background 66.59721056327747 current 105.4248682211424
Background 66.59721056327747 current 116.14981965014078
Background 66.59721056327747 current 112.56368456890912
Background 66.59721056327747 current 110.0350105225106
Background 66.59721056327747 current 108.99818159894322
Background 66.59721056327747 current 111.12874035100417
Background 66.59721056327747 current 109.03749538672778
Background 66.59721056327747 current 113.42952640604607
Background 66.59721056327747 current 111.136